## DSPy Demo
This notebook demonstrates the of DSPy framework for automatic prompt optimization.

In [ ]:
%cd ../

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup

In [ ]:
import json
import os
from pathlib import Path
from typing import Literal, Any, cast

import dspy
from dotenv import load_dotenv
from dspy.adapters.baml_adapter import BAMLAdapter
from sklearn.model_selection import train_test_split
import mlflow

from notebooks.utils.dtos import Merger, Acquisition, Other
from notebooks.utils.evaluation import validate_answer
from notebooks.utils.helpers import extract_first_n_sentences, read_data, get_gt_pydantic_model


Load environment with MlFlow and OpenAI keys.

In [ ]:
load_dotenv()

## MlFlow Setup

Run MlFlow server for logging traces and tracking optimization runs.

In [ ]:
MLFLOW_PORT = os.getenv("MLFLOW_PORT")

In [ ]:
!docker run -d -p {MLFLOW_PORT}:5000 -v $(pwd)/mlruns:/mlflow/mlruns -e MLFLOW_TRACKING_URI=http://0.0.0.0:5000 \
  --name mlflow-server ghcr.io/mlflow/mlflow:v3.7.0 mlflow server \
    --host 0.0.0.0 \
    --port 5000 \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root /mlflow/mlruns

In [ ]:
mlflow.set_tracking_uri(f"http://localhost:{MLFLOW_PORT}")

## DSPy

### Adapters
Adapters are basically middleware for communication with LLM. They support different way of interacting with LLM such as structured output.

In [ ]:
predictor_lm = dspy.LM(
    model="openrouter/google/gemini-2.5-flash-lite",
    api_base="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

dspy.configure(lm=predictor_lm, adapter=BAMLAdapter()) # type: ignore

Turn on MlFlow tracking with auto-logging for DSPy.

In [ ]:
mlflow.set_experiment("DSPy Demo")
mlflow.dspy.autolog(log_compiles=True, log_evals=True, log_traces_from_compile=True)

### Signatures
Signatures are declarative specification of the desired behavior that we expect from the LLM.

In [ ]:
class ClassifyArticle(dspy.Signature):
    """
    Analyze the news article and classify it as a merger or acquisition deal.
    - A merger is when two companies combine to form a new entity.
    - An acquisition is when one company takes over another.
    - If it mentions a potential, rumored or failed deal, classify it as "Other".
    """

    text: str = dspy.InputField()
    article_type: Literal["merger", "acquisition", "other"] = dspy.OutputField()


class ExtractMergerInfo(dspy.Signature):
    """
    Extract information about companies involved in a merger deal.
    - If the currency symbol is just "$", assume USD.
    """

    text: str = dspy.InputField()
    merger_info: Merger = dspy.OutputField()


class ExtractAcquisitionInfo(dspy.Signature):
    """
    Extract information about companies involved in an acquisition deal.
    - If the currency symbol is just "$", assume USD.
    """

    text: str = dspy.InputField()
    acquisition_info: Acquisition = dspy.OutputField()

### Modules
Modules are core building block in DSPy that combine signatures and logic. Modules are composable and therefore can be combined to form more complex systems.

In [ ]:
class Extract(dspy.Module):
    num_sentences: int = 5

    def __init__(self):
        self.classifier = dspy.Predict(ClassifyArticle)
        self.merger_extractor = dspy.Predict(ExtractMergerInfo)
        self.acquisition_extractor = dspy.Predict(ExtractAcquisitionInfo)

    def classify(self, text: str) -> str:
        text = extract_first_n_sentences(text, num_sentences=self.num_sentences)

        result = self.classifier(text=text)
        article_type : str = result.article_type # type: ignore

        return article_type


    def forward(self, text: str, article_id: int) -> Merger | Acquisition | Other:
        article_type = self.classify(text)

        if article_type == "merger":
            extracted_result = self.merger_extractor(text=text)
            merger_info : Merger = extracted_result.merger_info
            merger_info.article_id = article_id
            return merger_info
        
        elif article_type == "acquisition":
            extracted_result = self.acquisition_extractor(text=text)
            acquisition_info : Acquisition = extracted_result.acquisition_info
            acquisition_info.article_id = article_id
            return acquisition_info
        else:
            return Other(article_id=article_id, article_type="other")

## Data Preparation
Load articles about gold mining mergers and acquisitions.

In [ ]:
articles = read_data(Path("data/articles.json"))
print(f"Example of loaded article with id {articles[0]['id']}: \n{json.dumps(articles[0], indent=2)}")

Load ground truth about the articles which contains their classification and information extraction.

In [ ]:
ground_truths: list[dict[str, Any]] = read_data(Path("data/ground_truth.json"))
print(f"Example of loaded ground truth with id {ground_truths[0]['article_id']}: \n{json.dumps(ground_truths[0], indent=2)}")


Convert data to DSPy examples.

In [ ]:
NUM_SENTENCES = 5
dspy_examples: list[dspy.Example] = []
for article  in articles:
    ground_truth: dict[str, Any] = ground_truths[article["id"] - 1] # type: ignore
    gt_pydantic_model = get_gt_pydantic_model(ground_truth)
    example = dspy.Example(
        text=(
            article["title"] + "\n" + extract_first_n_sentences(article["content"], NUM_SENTENCES)
        ), 
        article_id=article["id"], 
        expected_output=gt_pydantic_model
    ).with_inputs("text", "article_id") # type: ignore
    dspy_examples.append(example)

Perform train-test split. For DSPy we actually do not need lot of training data.  <br>
In fact, using something **around 20-80 train-test split is quite standard**.

In [ ]:
train_set, test_set = cast(
    tuple[list[dspy.Example], list[dspy.Example]],
    train_test_split(dspy_examples, test_size=0.50, random_state=39)
)

print(f"Train set size: {len(train_set)} \nTest set size:  {len(test_set)}")

## Run Baseline DSPy Program

Instantiate and test out baseline DSPy program without any optimization.

In [ ]:
baseline_extract = Extract()

Execute the baseline module.

In [ ]:
extracted_article_1 = baseline_extract(text=train_set[0].text, article_id=train_set[0].article_id)

In [ ]:
print(f"Extracted article:\n{extracted_article_1.model_dump_json(indent=2)}")

Explore the underlying prompt. With `n=1` you will see extraction prompt. While with `n=2` you will see classification prompt.

In [ ]:
# dspy.inspect_history(n=1)

Store the baseline extractor as DSPy program in JSON format.

In [ ]:
BASELINE_PROGRAM_PATH = "programs/baseline_extractor.json"
baseline_extract.save(BASELINE_PROGRAM_PATH)

## Optimizations

### LabelFewShot Optimizer

In [ ]:
label_fewshot_optimizer = dspy.BootstrapFewShotWithRandomSearch(
    metric=validate_answer,
    max_bootstrapped_demos=5,
    max_labeled_demos=10,
    max_rounds=1,
    max_errors=1,
    num_candidate_programs=10,
)

Compile the `fewshot_extract` by running the optimizer.

In [ ]:
fewshot_extract = label_fewshot_optimizer.compile(baseline_extract, trainset=train_set)

Save the optimized program.

In [ ]:
FEWSHOT_PROGRAM_PATH = "programs/fewshot_extractor.json"
fewshot_extract.save(FEWSHOT_PROGRAM_PATH)

## Evaluation

In [ ]:
models_configs = {
    "baseline": {
        "program_path": BASELINE_PROGRAM_PATH,
    },
    "fewshot": {
        "program_path": FEWSHOT_PROGRAM_PATH,
    }
}

In [ ]:
for model_name, model_config in models_configs.items():
    extract_model = Extract()
    extract_model.load(model_config["program_path"])

    train_result = dspy.Evaluate(
        devset=train_set,
        metric=validate_answer,
        num_threads=1,
        display_progress=False,
        display_table=False,
    )(extract_model)

    test_result = dspy.Evaluate(
        devset=test_set,
        metric=validate_answer,
        num_threads=1,
        display_progress=False,
        display_table=False,
    )(extract_model)

    models_configs[model_name]["train_score"] = train_result["score"]
    models_configs[model_name]["test_score"] = test_result["score"]

In [ ]:
models_configs